#  Nearfield Ptychography

**Learning how to reconstruct a nearfield ptychography dataset with PtyPy.**

---

In [ ]:
import ptypy, h5py, os
import ptypy.utils as u

# This will import the HDF5Loader class
ptypy.load_ptyscan_module("hdf5_loader")

# This will import the GPU engines
ptypy.load_gpu_engines("cuda")  

# Root directory of tutorial data
tutorial_data_home = "/dls/science/groups/imaging/ptypy_tutorials/"

# Path to HDF5 file with raw data
dataset = "esrf_id16_AlNi_nearfield/S00084_data_bin1_newpos_2048x2048.h5"

# Absolute path to HDF5 file with raw data
path_to_data = os.path.join(tutorial_data_home, dataset)

with h5py.File(path_to_data, "r") as f:
    flatfield = f["flatfield"][:]

# Create parameter tree
p = u.Param()

# Set verbose level to interactive
p.verbose_level = "interactive"

# Scan label
p.run = "esrf_id16_AlNi_nearfield"

# Set io settings (no files saved)
p.io = u.Param()
p.io.rfile = None
p.io.autosave = u.Param(active=False)
p.io.autoplot = u.Param(active=False)
p.io.interaction = u.Param(active=False)

# Define the scan model
p.scans = u.Param()
p.scans.scan_00 = u.Param()
p.scans.scan_00.name = 'Full'

# Switch propagation to nearfield
p.scans.scan_00.propagation = "nearfield"

# Initial illumination (based on flatfield)
# p.scans.scan_00.illumination = u.Param()
# p.scans.scan_00.illumination.model = flatfield #"stxm"
# p.scans.scan_00.illumination.aperture = None
# p.scans.scan_00.illumination.propagation = u.Param()
# p.scans.scan_00.illumination.propagation.parallel = 0.019

p.scans.scan_00.illumination = u.Param()
p.scans.scan_00.illumination.model = "stxm"
p.scans.scan_00.illumination.aperture = None
p.scans.scan_00.illumination.propagation = u.Param()
p.scans.scan_00.illumination.propagation.parallel = 0.019

# Initial object
p.scans.scan_00.sample = u.Param()
p.scans.scan_00.sample.model = None
p.scans.scan_00.sample.diversity = None
p.scans.scan_00.sample.process = None

# Coherence parameters (modes)
p.scans.scan_00.coherence = u.Param()
p.scans.scan_00.coherence.num_probe_modes = 1
p.scans.scan_00.coherence.num_object_modes = 1

# Data loader
p.scans.scan_00.data = u.Param()
p.scans.scan_00.data.name = 'Hdf5Loader'
p.scans.scan_00.data.orientation = 3

p.scans.scan_00.data.intensities = u.Param()
p.scans.scan_00.data.intensities.file = path_to_data
p.scans.scan_00.data.intensities.key = 'data'

p.scans.scan_00.data.positions = u.Param()
p.scans.scan_00.data.positions.file = path_to_data
p.scans.scan_00.data.positions.slow_key = 'posy'
p.scans.scan_00.data.positions.slow_multiplier = 1
p.scans.scan_00.data.positions.fast_key = 'posx'
p.scans.scan_00.data.positions.fast_multiplier = 1

p.scans.scan_00.data.save = None   
p.scans.scan_00.data.auto_center = False  
p.scans.scan_00.data.load_parallel = 'data' 
p.scans.scan_00.data.center = 'fftshift' 
p.scans.scan_00.data.psize = 50.705  
p.scans.scan_00.data.distance = 0.019 
p.scans.scan_00.data.energy = 17.05

# Reconstruct using GPU-accelerated DM/ML engines
p.engines = u.Param()
p.engines.DM = u.Param()
p.engines.DM.name = 'DM_pycuda'
p.engines.DM.numiter = 2000
p.engines.DM.numiter_contiguous = 10
p.engines.DM.probe_support = None
p.engines.DM.clip_object = (.2, 1.1)
p.engines.DM.fourier_power_bound = 0.0
p.engines.DM.probe_update_start = 0

p.engines.ML = u.Param()
p.engines.ML.name = 'ML_pycuda'
p.engines.ML.numiter = 2000
p.engines.ML.numiter_contiguous = 10
p.engines.ML.ML_type = 'Gaussian'
p.engines.ML.floating_intensities = False
p.engines.ML.probe_support = 1.0
p.engines.ML.reg_del2 = True
p.engines.ML.reg_del2_amplitude = 0.005
p.engines.ML.scale_precond = False
p.engines.ML.probe_update_start = 0

# Run reconstruction
P = ptypy.core.Ptycho(p,level=5)

# Display the results
u.plot_recon_from_ptycho(P)

In [ ]:
u.plot_recon_from_ptycho(P, cropping=0.001, probe_channel="a")

In [ ]:
u.plot_recon_from_ptycho(P, probe_min=0, probe_max=200, probe_channel="a")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(flatfield, vmax=20000)
plt.colorbar()

In [ ]:
plt.imshow(P.diff.S["S0000"].data[0])